In [1]:
import numpy as np
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite
from os.path import join

In [2]:
data_dir = "/Users/phivri/Documents/GitHub/Biosustain/MoGeoModel"

WLmodel = cobra.io.read_sbml_model(join(data_dir,'Models/WLPathway_bifurcating.xml'))

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


To properly compare all results for the different substrate and product combinations, the model is constrained by the flux through CODH_ACS, so to equal Acetyl-CoA.

In [3]:
WLmodel.reactions.CODH_ACS.bounds = 1,1

Define carboxydotrophic and homoacetogenic media, Acetyl-CoA is constrained, so substrates are only absent or present and uptake rates are unconstrained

In [4]:
medium_carb = WLmodel.medium
medium_carb['EX_co_e'] = 1000
medium_carb['EX_co2_e'] = 0
medium_carb['EX_h2_e'] = 0

medium_hom = WLmodel.medium
medium_hom['EX_co_e'] = 0
medium_hom['EX_co2_e'] = 1000
medium_hom['EX_h2_e'] = 1000

In [5]:
objectives = pd.Series(["EX_ac_e","EX_etoh_e","EX_act_e","EX_but_e","EX_btoh_e","EX_btd_RR_e"])
formulas = pd.Series(["CH3COOH","C2H5OH","C3H6O","C4H7O2","C4H9OH","C4H10O2"], index=objectives)
Cmol =  pd.Series([2,2,3,4,4,4], index=objectives)

for x in objectives:
    print(WLmodel.reactions.get_by_id(x).name)
    WLmodel.objective = x
    with WLmodel:
        WLmodel.medium = medium_carb
        WLmodel.optimize()
        print(f" carb: {round(WLmodel.reactions.EX_co_e.flux,2)} CO + {round(WLmodel.reactions.EX_h2o_e.flux,2)} H2O + {round(WLmodel.reactions.EX_co2_e.flux,2)} CO2 + {round(WLmodel.slim_optimize(),2)} {formulas[x]} + {round(WLmodel.reactions.EX_h_e.flux,2)} H+ + {round(WLmodel.reactions.SK_atp_c.flux,2)} ATP")
        print("   ATP/Electron donor: ", round(-WLmodel.reactions.SK_atp_c.flux/WLmodel.reactions.EX_co_e.flux,2))
        print("   Product/Electron donor: ", round(-WLmodel.slim_optimize()*Cmol[x]/WLmodel.reactions.EX_co_e.flux,2))

    with WLmodel:
        WLmodel.medium = medium_hom
        WLmodel.optimize()
        print(f" hom: {round(WLmodel.reactions.EX_co2_e.flux,2)} CO2 + {round(WLmodel.reactions.EX_h2_e.flux,2)} H2 + {round(WLmodel.reactions.EX_h2o_e.flux,2)} H2O + {round(WLmodel.slim_optimize(),2)} {formulas[x]} + {round(WLmodel.reactions.EX_h_e.flux,2)} H+ + {round(WLmodel.reactions.SK_atp_c.flux,2)} ATP")
        print("   ATP/Electron donor: ", round(-WLmodel.reactions.SK_atp_c.flux/WLmodel.reactions.EX_h2_e.flux,2))
        print("   Product/Electron donor: ", round(-WLmodel.slim_optimize()*Cmol[x]/WLmodel.reactions.EX_h2_e.flux,2), "\n")

Acetate exchange
 carb: -4.0 CO + 0.0 H2O + 2.0 CO2 + 1.0 CH3COOH + -1.0 H+ + 2.0 ATP
   ATP/Electron donor:  0.5
   Product/Electron donor:  0.5
 hom: -2.0 CO2 + -4.0 H2 + 2.67 H2O + 1.0 CH3COOH + 0.33 H+ + 0.67 ATP
   ATP/Electron donor:  0.17
   Product/Electron donor:  0.5 

Ethanol exchange
 carb: -6.0 CO + -0.33 H2O + 4.0 CO2 + 1.0 C2H5OH + -2.67 H+ + 2.67 ATP
   ATP/Electron donor:  0.44
   Product/Electron donor:  0.33
 hom: -2.0 CO2 + -6.0 H2 + 3.67 H2O + 1.0 C2H5OH + -0.67 H+ + 0.67 ATP
   ATP/Electron donor:  0.11
   Product/Electron donor:  0.33 

Acetone exchange
 carb: -4.0 CO + -0.17 H2O + 2.5 CO2 + 0.5 C3H6O + -1.33 H+ + 1.33 ATP
   ATP/Electron donor:  0.33
   Product/Electron donor:  0.38
 hom: -1.5 CO2 + -4.0 H2 + 2.5 H2O + 0.5 C3H6O + 0.0 H+ + 0.0 ATP
   ATP/Electron donor:  0.0
   Product/Electron donor:  0.38 

Butyrate (n-C4:0) exchange
 carb: -5.0 CO + 0.17 H2O + 3.0 CO2 + 0.5 C4H7O2 + -1.67 H+ + 2.17 ATP
   ATP/Electron donor:  0.43
   Product/Electron donor:  

## Closer look, change x within [0:5] to view different product

In [6]:
x = 0

WLmodel.objective = objectives[x]

solution = WLmodel.optimize()

Builder(model = WLmodel, map_json = join(data_dir,'Models/EscherMaps/WLPathway_Bifurcating.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'EX_h2_e': -4.0, 'EX_h2o_e': 2.6666666666666665, 'EX_co2_e': -2.0, 'EX_co_e': 0.0, 'EX_…